In [1]:
import sys,math
import sympy as sym
import numpy as np
import os
from tqdm import tqdm
import pandas as pd
import time

In [2]:
bond_thresh = 1.2
rad2deg = 180.0 / math.pi
deg2rad = 1.0 / rad2deg

In [3]:
cov_rads = {  'H' : 0.37, 'C' : 0.77, 'O' : 0.73, 'N' : 0.75, 'F' : 0.71,
  'P' : 1.10, 'S' : 1.03, 'Cl': 0.99, 'Br': 1.14, 'I' : 1.33, 'He': 0.30,
  'Ne': 0.84, 'Ar': 1.00, 'Li': 1.02, 'Be': 0.27, 'B' : 0.88, 'Na': 1.02,
  'Mg': 0.72, 'Al': 1.30, 'Si': 1.18, 'K' : 1.38, 'Ca': 1.00, 'Sc': 0.75,
  'Ti': 0.86, 'V' : 0.79, 'Cr': 0.73, 'Mn': 0.67, 'Fe': 0.61, 'Co': 0.64,
  'Ni': 0.55, 'Cu': 0.46, 'Zn': 0.60, 'Ga': 1.22, 'Ge': 1.22, 'As': 1.22,
  'Se': 1.17, 'Kr': 1.03, 'X' : 0.00}

In [4]:
def get_file_string_array(file_name):
    try:
        file = open(file_name, "r")
    except IOError:
        print('Error: file (%s) not found!\n' % (file_name))
        sys.exit()
    lines = file.readlines()
    file.close()
    array = []
    for line in lines:
        array.append(line.split())
    return array

In [5]:
def get_geom(xyz_file_name):
    xyz_array = get_file_string_array(xyz_file_name)
    n_atoms = int(xyz_array[0][0])
    at_types = ['' for i in range(n_atoms)]
    coords = [[0.0 for j in range(3)] for i in range(n_atoms)]
    for i in range(n_atoms):
        at_types[i] = xyz_array[i+2][0]
        for j in range(3):
            coords[i][j] = float(xyz_array[i+2][j+1])
    geom = [at_types, coords]
    return geom,n_atoms

In [6]:
def get_r12(coords1, coords2):
    r2 = 0.0
    for p in range(3):
        r2 += (coords2[p] - coords1[p])**2
    r = math.sqrt(r2)
    return r

In [7]:
def get_u12(coords1, coords2):
    r12 = get_r12(coords1, coords2)
    u12 = [0.0 for p in range(3)]
    for p in range(3):
        u12[p] = (coords2[p] - coords1[p]) / r12
    return u12

In [8]:
def get_udp(uvec1, uvec2):
    udp = 0.0
    for p in range(3):
        udp += uvec1[p] * uvec2[p]
    udp = max(min(udp, 1.0), -1.0)
    return udp

In [9]:
def get_ucp(uvec1, uvec2):
    ucp = [0.0 for p in range(3)]
    cos_12 = get_udp(uvec1, uvec2)
    sin_12 = math.sqrt(1 - cos_12**2)
    ucp[0] = (uvec1[1]*uvec2[2] - uvec1[2]*uvec2[1]) / sin_12
    ucp[1] = (uvec1[2]*uvec2[0] - uvec1[0]*uvec2[2]) / sin_12
    ucp[2] = (uvec1[0]*uvec2[1] - uvec1[1]*uvec2[0]) / sin_12
    return ucp

In [10]:
def get_a123(coords1, coords2, coords3):
    u21 = get_u12(coords2, coords1)
    u23 = get_u12(coords2, coords3)
    dp2123 = get_udp(u21, u23)
    a123 = rad2deg * math.acos(dp2123)
    return a123

In [11]:
def get_t1234(coords1, coords2, coords3, coords4):
    u21 = get_u12(coords2, coords1)
    u23 = get_u12(coords2, coords3)
    u32 = get_u12(coords3, coords2)
    u34 = get_u12(coords3, coords4)
    u21c23 = get_ucp(u21, u23)
    u32c34 = get_ucp(u32, u34)
    dp = get_udp(u21c23, u32c34)
    sign = 2 * float(get_udp(u21c23, u34) < 0) - 1
    t1234 = rad2deg * sign * math.acos(dp)
    return t1234

In [12]:
def get_bond_graph(geom):
    at_types, coords = geom[0:2]
    n_atoms = len(at_types)
    bond_graph = [[] for i in range(n_atoms)]
    for i in range(n_atoms):
        covrad1 = cov_rads[at_types[i]]
        for j in range(i+1, n_atoms):
            covrad2 = cov_rads[at_types[j]]
            thresh = bond_thresh * (covrad1 + covrad2)
            r12 = get_r12(coords[i], coords[j])
            if (r12 < thresh):
                bond_graph[i].append(j)
                bond_graph[j].append(i)
    return bond_graph

In [13]:
def get_bonds(geom, bond_graph):
    at_types, coords = geom[0:2]
    n_atoms = len(at_types)
    bonds = []
    for i in range(n_atoms):      
        for a in range(len(bond_graph[i])):
            j = bond_graph[i][a]
            if (i < j):
                r12 = get_r12(coords[i], coords[j])
                bonds.append(r12)
    return bonds

In [14]:
def get_angles(geom, bond_graph):
    at_types, coords = geom[0:2]
    n_atoms = len(at_types)
    angles = []
    for j in range(n_atoms):
        n_jbonds = len(bond_graph[j])
        for a in range(n_jbonds):
            i = bond_graph[j][a]
            for b in range(a+1, n_jbonds):
                k = bond_graph[j][b]
                a123 = get_a123(coords[i], coords[j], coords[k])
                angles.append(a123)
    return angles

In [15]:
def get_torsions(geom, bond_graph):
    at_types, coords = geom[0:2]
    n_atoms = len(at_types)
    torsions = []
    for j in range(n_atoms):
        n_jbonds = len(bond_graph[j])
        for a in range(n_jbonds):
            k = bond_graph[j][a]
            if (k < j):
                continue
            n_kbonds = len(bond_graph[k])
            for b in range(n_jbonds):
                i = bond_graph[j][b]
                if (i == k):
                    continue
                for c in range(n_kbonds):
                    l = bond_graph[k][c]
                    if (l == j or l == i):
                        continue
                    t1234 = get_t1234(coords[i], coords[j], coords[k], coords[l])
                    torsions.append(t1234)
    return torsions

In [16]:
def make_func(bond1,angle1,torsion1):
    E_bond = 0
    E_theta = 0
    E_phi = 0
    E_phi2 = 0
    k_theta = 1
    k_b = 1
    k_phi = 1
    A = 1
    tau = 1
    b = sym.Symbol("b")
    phi = sym.Symbol("phi")
    theta = sym.Symbol("theta")
    for i in bond1:
        E_bond = E_bond + k_b*(b-i)**2
    for i in angle1:
        E_theta = E_theta + k_theta*(theta-i)**2
    for i in torsion1:
        E_phi = E_phi + k_phi*(phi-i)**2
    for i in range(1,7):
        for j in torsion1:
            E_phi2 = E_phi2 + A*(1+math.cos(math.degrees(i*tau + j)))
    E_total = E_theta+E_bond+E_phi+E_phi2;
    return E_total

In [17]:
def StD(f):
    b = sym.Symbol("b")
    phi = sym.Symbol("phi")
    theta = sym.Symbol("theta")
    x = [1000]
    y = [-200]
    z = [50]
    df_dx = sym.diff(f,b)
    df_dy = sym.diff(f,phi)
    df_dz = sym.diff(f,theta)
    g0 = [df_dx.subs([(b,x[0]),(phi,y[0]),(theta,z[0])]),df_dy.subs([(b,x[0]),(phi,y[0]),(theta,z[0])]),df_dz.subs([(b,x[0]),(phi,y[0]),(theta,z[0])])]
    d = [-x for x in g0]
    for i in range(0,8):
        I = [x[i],y[i],z[i]]
        t = sym.Symbol("t")
        g = f.subs([(b,x[i]+d[0]*t),(phi,y[i]+t*d[1]),(theta,z[i]+t*d[2])])
        dg_dt = sym.diff(g,t)
        t = sym.solve(dg_dt,t)
        t.append(0)
        x.append(I[0]+t[0]*d[0])
        y.append(I[1]+t[0]*d[1])
        z.append(I[2]+t[0]*d[2])
        g_o = [df_dx.subs([(b,x[i]),(phi,y[i]),(theta,z[i])]),df_dy.subs([(b,x[i]),(phi,y[i]),(theta,z[i])]),df_dz.subs([(b,x[i]),(phi,y[i]),(theta,z[i])])]
        i = i+1;
        g_1 = [df_dx.subs([(b,x[i]),(phi,y[i]),(theta,z[i])]),df_dy.subs([(b,x[i]),(phi,y[i]),(theta,z[i])]),df_dz.subs([(b,x[i]),(phi,y[i]),(theta,z[i])])]
        d=[-x for x in g_1];
    min1 = [ x[i], y[i], z[i], f.subs([(b,x[i]),(phi,y[i]),(theta,z[i])])]
    return min1

In [18]:
def CnG(f):
    b = sym.Symbol("b")
    phi = sym.Symbol("phi")
    theta = sym.Symbol("theta")
    x = [1000]
    y = [-200]
    z = [50]
    df_dx = sym.diff(f,b)
    df_dy = sym.diff(f,phi)
    df_dz = sym.diff(f,theta)
    g0 = [df_dx.subs([(b,x[0]),(phi,y[0]),(theta,z[0])]),df_dy.subs([(b,x[0]),(phi,y[0]),(theta,z[0])]),df_dz.subs([(b,x[0]),(phi,y[0]),(theta,z[0])])]
    d = [-x for x in g0]
    for i in range(0,8):
        I = [x[i],y[i],z[i]]
        t = sym.Symbol("t")
        g = f.subs([(b,x[i]+d[0]*t),(phi,y[i]+t*d[1]),(theta,z[i]+t*d[2])])
        dg_dt = sym.diff(g,t)
        t = sym.solve(dg_dt,t)
        t.append(0)
        x.append(I[0]+t[0]*d[0])
        y.append(I[1]+t[0]*d[1])
        z.append(I[2]+t[0]*d[2])
        g_o = [df_dx.subs([(b,x[i]),(phi,y[i]),(theta,z[i])]),df_dy.subs([(b,x[i]),(phi,y[i]),(theta,z[i])]),df_dz.subs([(b,x[i]),(phi,y[i]),(theta,z[i])])]
        i = i+1;
        g_1 = [df_dx.subs([(b,x[i]),(phi,y[i]),(theta,z[i])]),df_dy.subs([(b,x[i]),(phi,y[i]),(theta,z[i])]),df_dz.subs([(b,x[i]),(phi,y[i]),(theta,z[i])])]
        g_o_i = np.linalg.pinv(np.transpose([g_o]).astype(np.float64))
        beta = (g_1*g_o_i)**2
        d = np.transpose([-x for x in g_1]) + beta*np.transpose(d)
        d = d[0]
    min1 = [ x[i], y[i], z[i], f.subs([(b,x[i]),(phi,y[i]),(theta,z[i])])]
    return min1

In [19]:
def minimize(file_name):
    start = time.time()
    geom,no = get_geom(file_name)
    bond_graph = get_bond_graph(geom)
    bonds = get_bonds(geom, bond_graph)
    angles = get_angles(geom, bond_graph)
    torsions = get_torsions(geom, bond_graph)
    f = make_func(bonds,angles,torsions)
    end = time.time()
    start1 = time.time()
    mini1 = StD(f)
    end1 = time.time()
    start2 = time.time()
    mini2 = CnG(f)
    end2 = time.time()
    time1 = end-start
    time2 = end1-start1
    time3 = end2-start2
    return mini1, mini2, no, time1, time2, time3

In [ ]:
list_files = os.listdir("XYZ files")
molecule = []
minimum_std = []
minimum_cng = []
n_atoms = []
p_time = []
s_time = []
c_time = []
for i in tqdm(range(len(list_files)), desc = 'PROCESSING...'):
    path = "C:\\Users\\admin\\Desktop\\StD,CnG\\XYZ files\\"
    file = path + list_files[i]
    min1,min2,no,timep, times, timec = minimize(file)
    c_file = list_files[i]
    molecule.append(c_file[:len(c_file)-4])
    minimum_std.append(min1)
    minimum_cng.append(min2)
    n_atoms.append(no)
    p_time.append(timep)
    s_time.append(times)
    c_time.append(timec)

PROCESSING...:  73%|███████▎  | 199/274 [58:05<08:32,  6.84s/it]   

In [ ]:
Compilation_Min = pd.DataFrame({'Molecule Name' : molecule,
                                'Min Binding eng.-StD' : minimum_std,
                                'Min Binding eng.-CnG' : minimum_cng, 
                                'No.of atoms in molec.': n_atoms,
                                'Processing time': p_time,
                                'STD time' : s_time,
                                'CNG time' : c_time},
                                columns=['Molecule Name','Min Binding eng.-StD', 'Min Binding eng.-CnG','No.of atoms in molec.','Processing time','STD time','CNG time'])

In [ ]:
writer = pd.ExcelWriter('output.xlsx')
Compilation_Min.to_excel(writer)
writer.save()
print('DataFrame is written successfully to Excel File.')